# Big Exp 3 Alternative 1


### Authors: Quentin Duchemin & Yohann De Castro

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from inverse_map import inverse_map, train_network
import torch
import scipy
import scipy as sc
import scipy.stats
from PSIGLL import *

In [2]:
n,p = 10000,8
lamb = 2

d = 2
theta = 10*np.ones(d)
truetheta = np.zeros(p)
truetheta[:d] = theta

np.random.seed(1)

rescale = 1 #10 * np.sqrt(n)
lamb *= rescale * lamb


X = rescale * np.random.normal(0,1,(n,p))
#X /= np.tile(np.linalg.norm(X,axis=0),(n,1))
# proj = X[:,:d] @ np.linalg.inv(X[:,:d].T @ X[:,:d]) @ X[:,:d].T
# X[:,d:] = (np.eye(n)-proj) @ X[:,d:] 


matXtrue = X[:,:d]

sig = sigmoid(matXtrue @ theta)
yobs = np.random.rand(n) <= sig
model = LogisticRegression(C = 1/lamb, penalty='l1', solver='liblinear', fit_intercept=False)
model.fit(X, yobs)
theta_obs = model.coef_[0]
M = np.where( np.abs(theta_obs) > 1e-5)[0]
print('Selected support: ', M)
print('Size selected support: ', len(M))


# definition of the null distribution
thetanull = truetheta #np.zeros(p)
signull = sigmoid(X @ thetanull)

Selected support:  [0 1 3 4 5 6]
Size selected support:  6


In [3]:
SEI_sampling = True
if SEI_sampling:
    states = SEI_by_sampling(sig, X, lamb, M, nb_ite=10000000)
    if (np.abs(signull - sig)>1e-3).any():
        statesnull = SEI_by_sampling(signull, X, lamb, M, nb_ite=10000000)
    else:
        statesnull = np.copy(states)
else:
    probasalt, EM_states = true_conditional_distribution(theta_obs,X,yobs,lamb,truetheta,conditioning_signs=False)
    probasnull, EM_statesnull = true_conditional_distribution(theta_obs,X,yobs,lamb,thetanull,conditioning_signs=False,states=states)
    idxs = np.random.choice([i for i in range(len(EM_states))], size=300, p=probasalt)
    states = [EM_states[i] for i in idxs]
    idxs_null = np.random.choice([i for i in range(len(EM_statesnull))], size=300, p=probasnull)
    statesnull = [EM_statesnull[i] for i in idxs_null]
print('Number of states in the selection event: ', len(states))
np.save('states_BE3_alt_1e1.npy', states)

  0%|          | 0/10000000 [00:00<?, ?it/s]

KeyboardInterrupt: 

##### Computing $\bar \pi ^{\pi ^0}$ and training the NN aiming at computing $\Psi=\Xi^{-1}$

In [ ]:
n,p = np.shape(X)
matXtrue = X[:,M]
tildeGN_12, barpi = params_saturated(signull, matXtrue, statesnull)
#net, loss_values = train_network(matXtrue/rescale**2,max(1,truetheta[0]),nb_epochs=100,lrstart=0.1,lrdecay_step=80)

In [ ]:
lspvals_selec, lspvals_sat = pval_SIGLE(states, X, M, barpi, net=None, use_net_MLE=False, l2_regularization=10000)

In [ ]:
lists_pvalues = [lspvals_selec, lspvals_sat]
names = ['SIGLE Selected', 'SIGLE Saturated']
plot_cdf_pvalues(lists_pvalues, names, name_figsave='alt_1e1_be3.png')


##### Checking assumption of the Conditional CLT

In [ ]:
coarse_upper_bound, upper_bound = upper_bound_condition_CCLT(states[:1000],X,barpi,tildeGN_12,M)
print("Coarse Upper bound (Theorem statement): ", coarse_upper_bound)
print("Upper bound (weaker requirement obtained in the proof): ", upper_bound)
print("-> Both should tend to 0 as n grows.")

In [ ]:
matXtrue = X[:,M]
rho = matXtrue.T @ barpi.T
try:
    tildetheta = net(torch.from_numpy(rho.T).float())
    tildetheta = tildetheta.detach().numpy()
except:
    model = LogitRegressionContinuous()
    model.fit(matXtrue, barpi)
    tildetheta = model.coef_
tildeGN = matXtrue.T @ np.diag(barpi*(np.ones(n)-barpi)) @ matXtrue
usvd,s,vt = np.linalg.svd(tildeGN)
tildeGN_12 = usvd @ np.diag(1/np.sqrt(s)) @ vt
GNtilde = matXtrue.T @ np.diag(sigmoid1(matXtrue @ tildetheta)) @ matXtrue
VN = tildeGN_12 @ GNtilde

lsstat = []
u  = np.random.normal(0,1,len(M))
u /= np.linalg.norm(u)
u = np.zeros(len(M))
u[0] = 1
lspvals_selec = []
for i in range(len(states)):
    y = np.array(states[i])
    # selected
    model = LogisticRegression(C=100000000, solver='liblinear', fit_intercept=False)
    model.fit(matXtrue, y)
    theta = model.coef_[0]
    stat = u.T @ VN @ (theta - tildetheta)
    stat = u.T @ tildeGN_12 @ matXtrue.T @ (y-barpi)
    lsstat.append(stat)
    stat2 = np.linalg.norm( VN @ (theta - tildetheta))**2
    lspvals_selec.append(1-scipy.stats.chi2.cdf(stat2, len(M)))
a= plt.hist(lsstat, density=True, alpha=0.2, bins=30)
b = np.random.normal(0,1,500)
c = plt.hist(b,density=True, bins=30, alpha=0.2, label='true gaussian')
plt.legend()

In [ ]:
a = plt.hist(lspvals_selec)